In [ ]:
# import libraries
import os
import geopandas as gpd

ff_api_key = 'your_ff_api_key_here'  # replace with your FastFuels API key
os.environ["FASTFUELS_API_KEY"] = ff_api_key

from fastfuels_sdk.domains import Domain
from fastfuels_sdk.features import Features
from fastfuels_sdk.grids import TopographyGridBuilder
from fastfuels_sdk.inventories import Inventories
from fastfuels_sdk.grids import TreeGridBuilder
from fastfuels_sdk.grids import SurfaceGridBuilder
from fastfuels_sdk.grids import Grids

In [2]:
# function that will create domain and grids for fast fuels
def create_domain_and_grids(boundary, site_name, product, 
                            version, interpolation_method, 
                            curing_live_herbaceous, curing_live_woody, 
                            groups, feature_masks, remove_non_burnable, 
                            uniform_fuel_moisture_value, grid_data_zip_path,
                            tree_inventory = False, tree_inventory_path = None,
                            horizontal_resolution=2.0, vertical_resolution=1.0):

    """
    Create a domain and associated grids for a given site in fastfuels.

    Parameters:
    boundary (GeoDataFrame): Geospatial boundary of the site.
    site_name (str): Name of the site.
    product (str): Product type for fuel load data.
    version (str): Version of the product.
    interpolation_method (str): Method for interpolating data.
    curing_live_herbaceous (float): Curing percentage for live herbaceous fuels.
    curing_live_woody (float): Curing percentage for live woody fuels.
    groups (list): List of groups for fuel data.
    feature_masks (dict): Dictionary of feature masks.
    remove_non_burnable (bool): Flag to remove non-burnable areas.
    uniform_fuel_moisture_value (float): Uniform fuel moisture value.
    grid_data_zip_path (str): Path to save grid_data
    tree_inventory (bool): Flag to make tree inventory out of csv file
    tree_inventory_path (str): path to tree inventory csv
    horizontal_resolution (float, optional): Horizontal resolution for the domain. Default is 2.0.
    vertical_resolution (float, optional): Vertical resolution for the domain. Default is 1.0.

    Returns:
    dict: A dictionary containing the created domain, features, grids, feature grid, topography grid, tree inventory, tree grid, and surface grid.
    """

    
    # Create domain
    domain = Domain.from_geodataframe(
        geodataframe=boundary,
        name=site_name,
        description=f"domain of {site_name}",
        horizontal_resolution=horizontal_resolution,  # Use the argument
        vertical_resolution=vertical_resolution       # Use the argument
    )
    print(f"Created domain with ID: {domain.id}")

    # Initialize Features for our domain
    features = Features.from_domain_id(domain.id)

    # Initialize grids for our domain
    grids = Grids.from_domain_id(domain.id)

    # Create features from OpenStreetMap
    road_feature = features.create_road_feature_from_osm()
    water_feature = features.create_water_feature_from_osm()

    # Wait for features to be ready
    road_feature.wait_until_completed()
    water_feature.wait_until_completed()

    print("created road and water")

    # Create a feature grid with roads and water bodies
    feature_grid = grids.create_feature_grid(
        attributes=["road", "water"]
    )
    feature_grid.wait_until_completed()
    print("created feature grid")

    topography_grid = (
        TopographyGridBuilder(domain_id=domain.id)
        .with_elevation_from_3dep()
        .with_slope_from_3dep()
        .with_aspect_from_3dep()
        .build()
    )

    topography_grid.wait_until_completed()
    print("created topo")

    # if tree inventory flag is true (you want to use a csv file as the tree inventory)
    if tree_inventory:
        tree_inventory = Inventories.from_domain_id(
            domain.id
        ).create_tree_inventory_from_file_upload(
            # feature_masks=feature_masks,
            file_path=(tree_inventory_path)
        )
        tree_inventory.wait_until_completed()
        print("created tree")

    # else Create tree inventory
    else:
        tree_inventory = Inventories.from_domain_id(
            domain.id
        ).create_tree_inventory_from_treemap(
            feature_masks=feature_masks
        )
        tree_inventory.wait_until_completed()
        print("created tree")
    

    # Create tree grid
    tree_grid = (
        TreeGridBuilder(domain_id=domain.id)
        .with_bulk_density_from_tree_inventory()
        .with_spcd_from_tree_inventory()
        .build()
    )
    tree_grid.wait_until_completed()

    print("created treegrid")

    # Create surface grid
    surface_grid = (
        SurfaceGridBuilder(domain_id=domain.id)
        .with_fuel_load_from_landfire(
            product=product,
            version=version,
            interpolation_method=interpolation_method,
            curing_live_herbaceous=curing_live_herbaceous,
            curing_live_woody=curing_live_woody,
            groups=groups,
            feature_masks=feature_masks,
            remove_non_burnable=remove_non_burnable,
        )
        .with_fuel_depth_from_landfire(
            product=product,
            version=version,
            interpolation_method=interpolation_method,
            feature_masks=feature_masks,
            remove_non_burnable=remove_non_burnable,
        )
        .with_uniform_fuel_moisture(
            value=uniform_fuel_moisture_value,
            feature_masks=feature_masks
        )
        .with_savr_from_landfire(
            product=product,
            version=version,
            interpolation_method=interpolation_method,
            groups=groups,
            feature_masks=feature_masks,
            remove_non_burnable=remove_non_burnable
        )
        .build()
    )
    surface_grid.wait_until_completed()
    print("created surface")

    # Create the export
    export = grids.create_export("zarr") #"zarr" or "QUICFire"
    
    # Wait for the export to complete
    export.wait_until_completed()
    print('exported')
    
    # Save to a file
    export.to_file(grid_data_zip_path)


    return {
        "domain": domain,
        "features": features,
        "grids": grids,
        "feature_grid": feature_grid,
        "topography_grid": topography_grid,
        "tree_inventory": tree_inventory,
        "tree_grid": tree_grid,
        "surface_grid": surface_grid
    }



In [ ]:
# set the parameters
boundary = gpd.read_file('/path/to/boundary/file') 
site_name = 'sitename'
product="FBFM40"
version='2022'
interpolation_method='zipper'
curing_live_herbaceous=0.25
curing_live_woody=0.1
groups = ['oneHour']
feature_masks=["road", "water"]
remove_non_burnable=["NB1", "NB2"]
uniform_fuel_moisture_value=0.15
grid_data_zip_path = '/path/to/.zip/file' #fuel-array.zip - this is where you want to export the zarr file as 
tree_inventory = False
tree_inventory_path = None
horizontal_resolution=2.0
vertical_resolution=1.0


In [4]:
#create treelist from fastfuels
domaingrid = create_domain_and_grids(boundary, 
                                     site_name, 
                                     product=product, 
                                     version=version, 
                                     interpolation_method=interpolation_method, 
                                     curing_live_herbaceous=curing_live_herbaceous, 
                                     curing_live_woody=curing_live_woody, 
                                     groups = groups, 
                                     feature_masks = feature_masks, 
                                     remove_non_burnable = remove_non_burnable, 
                                     uniform_fuel_moisture_value=uniform_fuel_moisture_value, 
                                     grid_data_zip_path = grid_data_zip_path,
                                     tree_inventory = tree_inventory, 
                                     tree_inventory_path = tree_inventory_path,
                                     horizontal_resolution=horizontal_resolution, 
                                     vertical_resolution=vertical_resolution)

Created domain with ID: 4c8434a1c63d41df91892047ceac3517
created road and water
created feature grid
created topo
created tree
created treegrid
created surface
exported
